# Случайные деревья
Идея: проверить значимость переменных "забил предыдущий", "забил два предыдущих" и тд

### TO-DO list
* Попробовать другой критерий расщепления (энтропийный)
* Поиграть с параметрами случайного леса (помимо максимальной глубины и количества деревьев)

In [62]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier # случайный лес
from sklearn.model_selection import KFold # k-fold кросс-валидация
from sklearn.metrics import roc_auc_score # AUC для оценки качества модели
from sklearn.metrics import confusion_matrix # для определения FP, FN, TP, TN
from sklearn.model_selection import train_test_split # случайное разделение выборки

import matplotlib.pyplot as plt
%matplotlib inline

In [63]:
raw = pd.read_csv('curry1415.csv', header=0)

In [64]:
# Интересно, есть ли разница для алгоритма между переменными "координаты" и "расстояние" + "угол"?
# добавим более точную переменную расстояния
raw['dist'] = np.sqrt(raw['x']**2 + raw['y']**2)

# введем угол броска

loc_x_zero = raw['x'] == 0
raw['angle'] = np.array([0]*len(raw))
raw['angle'][~loc_x_zero] = np.arctan(raw['y'][~loc_x_zero] / raw['x'][~loc_x_zero])
raw['angle'][loc_x_zero] = np.pi / 2 

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [65]:
# единая переменная оставшегося времени
raw2=raw.copy()
raw['remaining_time'] = raw['minutes_remaining'] * 60 + raw['seconds_remaining']

In [66]:
# дропаем ненужные переменные
todrop = ['name', 'team_name', 'game_date', 'season', 'espn_player_id', 'team_id', \
          'espn_game_id', 'minutes_remaining', 'seconds_remaining', 'shot_distance', 'x', 'y', 'defender_name']

for i in todrop:
    raw = raw.drop(i, 1)

In [67]:
# создаем дамми-переменные из категориальных
categorical_vars = ['action_type', 'shot_type', 'opponent', 'period']

for i in categorical_vars:
    raw = pd.concat([raw, pd.get_dummies(raw[i], prefix=i)], 1)
    raw = raw.drop(i, 1)

In [68]:
# делим выборку на объясняющие переменные и таргетируемую
train = raw.drop('shot_made_flag', 1)
train_y = raw['shot_made_flag']

## Рандомное разделение выборки
just for fun попробуем рандомно разделить выборку и  прогнать алгоритм случайного леса. 

In [130]:
X_train, X_test, y_train, y_test = train_test_split(train, train_y, test_size=0.3, random_state=0)

In [131]:
rfc = RandomForestClassifier(n_estimators=150, max_depth=20)
rfc.fit(X_train, y_train)
pred=rfc.predict(X_test)

In [132]:
# Найдем самые простые оценки качества модели
a=confusion_matrix(y_test, pred)
acc=(a[0][0]+a[1][1])/len(y_test)
pr=a[1][1]/(a[1][1]+a[0][1])
rc=a[1][1]/(a[1][1]+a[1][0])
fmera=2*pr*rc/(pr+rc)
alg1=[acc, pr, rc, fmera]
print(acc, pr, rc, fmera)

0.593632958801 0.644230769231 0.483754512635 0.552577319588


Результаты не очень хорошие. Алгоритм плохо предсказывает действительность

## Оптимизация параметров леса с k-fold кросс-валидацией

In [72]:
scores_n = []
range_n = [10, 50, 100, 150, 250, 500]
kf = KFold(n_splits=10)
for n in range_n:    
    rfc = RandomForestClassifier(n_estimators=n)
    for train_k, test_k in kf.split(train):
        rfc.fit(train.iloc[train_k], train_y.iloc[train_k])
        pred = rfc.predict(train.iloc[test_k])
        a=confusion_matrix(train_y[test_k], pred)
        pr=a[1][1]/(a[1][1]+a[0][1])
        rc=a[1][1]/(a[1][1]+a[1][0])
        fmera=2*pr*rc/(pr+rc)
    scores_n.append(fmera)
print(scores_n)

[0.52972972972972976, 0.60999999999999999, 0.6133333333333334, 0.60909090909090913, 0.60633484162895934, 0.59907834101382496]


Таким образом, судя по F-мере, лучшее число деревьев равно 100

In [73]:
scores_m = []
range_m = [2,4,8,10,20,50]
kf = KFold(n_splits=10)
for m in range_m:
    rfc = RandomForestClassifier(max_depth=m, n_estimators=100)
    for train_k, test_k in kf.split(train):
        rfc.fit(train.iloc[train_k], train_y.iloc[train_k])
        pred = rfc.predict(train.iloc[test_k])
        a=confusion_matrix(train_y[test_k], pred)
        pr=a[1][1]/(a[1][1]+a[0][1])
        rc=a[1][1]/(a[1][1]+a[1][0])
        fmera=2*pr*rc/(pr+rc)
    scores_m.append(fmera)
print(scores_m)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: RuntimeWarning: invalid value encountered in double_scalars


[0.34042553191489361, 0.47311827956989244, 0.54450261780104714, 0.5490196078431373, 0.57943925233644866, 0.60747663551401865]


Судя по F-мере, лучший параметр максимальной глубины деревьев равен 20

In [74]:
# Критерий джини помогает нам определить наиболее значимые переменные при классификации
importance=rfc.feature_importances_
cols = train.columns.values
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': importance,
    })

In [75]:
# отсортируем значимость по убыванию
sorted_imp = feature_dataframe.sort_values(['Random Forest feature importances'], ascending=0)
sorted_imp.index=range(len(feature_dataframe))

In [76]:
# первые 20 значимых переменных
sorted_imp.head(20)

,Random Forest feature importances,features
0,0.121885,dist
1,0.099762,angle
2,0.097896,defender_distance
3,0.097154,shot_clock
4,0.096431,remaining_time
5,0.086470,touch_time
6,0.055579,dribbles
7,0.046945,action_type_Jump Shot
8,0.015243,period_3
9,0.013323,period_1


Получилось достаточно интуитивно. Больше всего на определение попадания/промаха влияют: расстояние до кольца, угол броска, оставшееся время в периоде, оставшееся время на атаку и расстояние до ближайшего защитника

## Введение новых переменных
Предположим, что броски зависимы друг от друга, и введем дамми-переменные "Забил i последних бросков подряд"

In [77]:
# отсортируем по времени
sorted_raw = raw2.sort_values(['espn_game_id', 'period', 'minutes_remaining', 'seconds_remaining'], ascending=[1, 1, 0, 0])
sorted_raw.index=range(len(sorted_raw))

In [78]:
sorted_raw['previous1'] = np.zeros(len(sorted_raw)) 

for i,row in enumerate(sorted_raw[1:].iterrows()):
    if i>0:
        if sorted_raw.loc[i,'espn_game_id'] == sorted_raw.loc[i-1,'espn_game_id']:
            sorted_raw.loc[i,'previous1'] = sorted_raw.loc[i-1,'shot_made_flag']

In [79]:
sorted_raw['previous2'] = np.zeros(len(sorted_raw)) 

for i,row in enumerate(sorted_raw[1:].iterrows()):
    if i>1:
        if sorted_raw.loc[i,'espn_game_id'] == sorted_raw.loc[i-1,'espn_game_id'] == sorted_raw.loc[i-2,'espn_game_id']:
            if sorted_raw.loc[i-1,'shot_made_flag']==sorted_raw.loc[i-2,'shot_made_flag']==1:
                sorted_raw.loc[i,'previous2'] = raw.loc[i-1,'shot_made_flag']

In [80]:
sorted_raw['previous3'] = np.zeros(len(sorted_raw)) 

for i,row in enumerate(sorted_raw[1:].iterrows()):
    if i>2:
        if sorted_raw.loc[i,'espn_game_id'] == sorted_raw.loc[i-1,'espn_game_id'] == sorted_raw.loc[i-2,'espn_game_id']== sorted_raw.loc[i-3,'espn_game_id']:
            if sorted_raw.loc[i-1,'previous2']==sorted_raw.loc[i,'previous1']==1:
                sorted_raw.loc[i,'previous3'] = raw.loc[i-1,'shot_made_flag']

In [81]:
sorted_raw['previous1']=sorted_raw['previous1'].astype(int)
sorted_raw['previous2']=sorted_raw['previous2'].astype(int)
sorted_raw['previous3']=sorted_raw['previous3'].astype(int)

Теперь почистим выборку, как мы делали это до этого

In [82]:
sorted_raw['remaining_time'] = sorted_raw['minutes_remaining'] * 60 + sorted_raw['seconds_remaining']
# дропаем ненужные переменные
todrop = ['name', 'team_name', 'game_date', 'season', 'espn_player_id', 'team_id', \
          'espn_game_id', 'minutes_remaining', 'seconds_remaining', 'shot_distance', 'x', 'y', 'defender_name']

for i in todrop:
    sorted_raw = sorted_raw.drop(i, 1)

# создаем дамми-переменные из категориальных
categorical_vars = ['action_type', 'shot_type', 'opponent', 'period']
# categorical_vars = ['action_type', 'shot_type', 'opponent', 'period', 'previous1', 'previous2', 'previous3']
for i in categorical_vars:
    sorted_raw = pd.concat([sorted_raw, pd.get_dummies(sorted_raw[i], prefix=i)], 1)
    sorted_raw = sorted_raw.drop(i, 1)
# делим выборку на объясняющие переменные и таргетируемую
trainn = sorted_raw.drop('shot_made_flag', 1)
trainn_y = sorted_raw['shot_made_flag']

In [126]:
X_train, X_test, y_train, y_test = train_test_split(trainn, trainn_y, test_size=0.3, random_state=0)
rfc = RandomForestClassifier(n_estimators=150, max_depth=20)
rfc.fit(X_train, y_train)
pred=rfc.predict(X_test)
# Найдем самые простые оценки качества модели
a=confusion_matrix(y_test, pred)
acc=(a[0][0]+a[1][1])/len(y_test)
pr=a[1][1]/(a[1][1]+a[0][1])
rc=a[1][1]/(a[1][1]+a[1][0])
fmera=2*pr*rc/(pr+rc)
alg2=[acc, pr, rc, fmera]
print(acc, pr, rc, fmera)

0.644194756554 0.666666666667 0.566037735849 0.612244897959


В целом, показатели улучшились. Неужели введение дополнительных переменных дает лучший результат? Посмотрим, насколько значимы нововведенные параметры

In [127]:
importance=rfc.feature_importances_
cols = trainn.columns.values
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': importance,
    })

In [128]:
sorted_impp = feature_dataframe.sort_values(['Random Forest feature importances'], ascending=0)
sorted_impp.index=range(len(feature_dataframe))

In [129]:
sorted_impp.head(20)

,Random Forest feature importances,features
0,0.114100,dist
1,0.100171,remaining_time
2,0.096951,angle
3,0.086932,shot_clock
4,0.085949,defender_distance
5,0.084811,touch_time
6,0.054195,dribbles
7,0.037231,action_type_Jump Shot
8,0.016908,previous1
9,0.015360,period_3


Прогресс налицо! Для случайного леса параметры "забил предыдущий" и "забил два предыдущих" стоят на 9 и 17 местах по важности. К сожалению, интерпретировать меру важности сложно. Нужно проверить на других алгоритмах

**Пометка:** после увеличения максимальной глубины важность "забил два предыдущих" упала. Почему?

In [133]:
# разница между acc, pr, rc, fmera
np.array(alg2)-np.array(alg1)

array([ 0.0505618 ,  0.0224359 ,  0.08228322,  0.05966758])

Количество правильных ответов растет на 5%, точность на 2%, полнота на 8% и F-мера на 6%